In [2]:
import sys
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
from tqdm import tqdm
from eeSAR.s1 import s1_collection, s1_timescan
from datetime import datetime as dt, timedelta
import ee
ee.Initialize()

In [3]:
gdf = gpd.read_file('/home/vollrath/Indonesia_sm/ground_truth_sm.gpkg')
len(gdf)

30316

In [ ]:
def addAux(image):
    
    def addGLDAS(image):

        def set_resample(image):
            """ Set resampling of the image to bilinear"""
            return image.resample()
        
        def add_date_difference(image):

            return image.set(
                'dateDist',
                ee.Number(image.get('system:time_start')).subtract(t.millis()).abs()
            )


        t = image.date()
        fro = t.advance(ee.Number(-30), 'days')
        #to = t.advance(ee.Number(10), 'days')

        gldas = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
            .select('SoilMoi0_10cm_inst') \
            .filterBounds(image.geometry()) \
            .map(set_resample)\
            
        gldas_stat = gldas.reduce(
                ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True)
            ).rename('gldas_mean', 'gldas_stddev')
        
        gldas = gldas.filterDate(fro, t).map(add_date_difference)
            
        sm_gldas = gldas.sort('dateDist').first().rename('sm_1')

        gldas_3day = gldas.filterDate(t.advance(ee.Number(-3), 'days'), t)
        gldas_3day = gldas_3day.sum().divide(gldas_3day.count()).rename('sm_3')

        gldas_7day = gldas.filterDate(t.advance(ee.Number(-7), 'days'), t)
        gldas_7day = gldas_7day.sum().divide(gldas_7day.count()).rename('sm_7')

        gldas_30day = gldas.sum().divide(gldas.count()).rename('sm_30')

        return image.addBands(gldas_stat).addBands(sm_gldas).addBands(gldas_3day).addBands(gldas_7day).addBands(gldas_30day)
    
    
    def addGPM(image):

        def set_resample(image):
            """ Set resampling of the image to bilinear"""
            return image.resample()

        def add_date_difference(image):

            return image.set(
                'dateDist',
                ee.Number(image.get('system:time_start')).subtract(t.millis()).abs()
            )

        t = image.date()
        # t = ee.Date(feature.get('date').getInfo()['value'])
        fro = t.advance(ee.Number(-30), 'days')

        gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
            .filterBounds(image.geometry()) \
            .filterDate(fro, t) \
            .select('HQprecipitation') \
            .map(add_date_difference) \
            .map(set_resample)


        gpm_closest = gpm.filterDate(t.advance(ee.Number(-1), 'days'), t)  
        gpm_closest = gpm_closest.sum().divide(gpm_closest.count()).rename('precipitation')

        gpm_3day = gpm.filterDate(t.advance(ee.Number(-3), 'days'), t)
        gpm_3day = gpm_3day.sum().divide(gpm_3day.count()).rename('prec_3')

        gpm_7day = gpm.filterDate(t.advance(ee.Number(-7), 'days'), t)
        gpm_7day = gpm_7day.sum().divide(gpm_7day.count()).rename('prec_7')

        gpm_30day = gpm.sum().divide(gpm.count()).rename('prec_30')

        return image.addBands(gpm_closest).addBands(gpm_3day).addBands(gpm_7day).addBands(gpm_30day)
    
    
    def addSRTM(image):

        srtm = ee.Image('USGS/SRTMGL1_003').resample()
        aspect = ee.Terrain.aspect(srtm).rename('aspect')
        slope = ee.Terrain.slope(srtm).rename('slope')

        return image.addBands(srtm.select('elevation').addBands(aspect).addBands(slope))
    
    
    def addGlobCover(image):
        
        lc = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3")
        return image.addBands(lc)
    
    def addTScans(image):
    
        def toLn(image):

            ln = image.select(['VV', 'VH']).log().rename(['VV', 'VH'])
            return image.addBands(ln, None, True)

        def toLin(image):

            lin = ee.Image(10).pow(image.select(['VV', 'VH']).divide(10)).rename(['VV', 'VH'])
            return image.addBands(lin, None, True)

        track_nr = image.get('relativeOrbitNumber_start')
        orbit = image.get('orbitProperties_pass').getInfo()

        tSeries = s1_collection.create(
            region=image.geometry(),
            orbits=[orbit],
            start_date='2014-01-01',
            end_date='2021-01-01',
            add_ratio=False,
            add_ND_ratio=False,
            speckle_filter='NONE',
            radiometric_correction='TERRAIN',
            slope_correction_dict={'model': 'surface', 'dem': 'USGS/SRTMGL1_003', 'buffer': 50},
            db=False,
            outlier_removal='AGGRESSIVE'
            ) \
            .filterMetadata('relativeOrbitNumber_start', 'equals', track_nr)

        # create combined reducer
        reducer = ee.Reducer.mean() \
            .combine(ee.Reducer.stdDev(), '', True) \
            .combine(ee.Reducer.percentile([5, 95]), '', True)

        # create log timescan (k variables)
        tScanLn = tSeries.map(toLn).select(['VV', 'VH']).reduce(reducer).select(
        ['VV_mean', 'VV_stdDev', 'VV_p5', 'VV_p95', 'VH_mean', 'VH_stdDev', 'VH_p5', 'VH_p95'],
        ['kVV_mean', 'kVV_stdDev', 'kVV_p5', 'kVV_p95', 'kVH_mean', 'kVH_stdDev', 'kVH_p5', 'kVH_p95'])

        # creeate linear timescan
        tScanLin = tSeries.map(toLin).select(['VV', 'VH']).reduce(reducer)

        return image.addBands(tScanLn).addBands(tScanLin)
    
    #image = addGPM(image)
    #image = addGLDAS(image)
    image = addSRTM(image)
    image = addGlobCover(image)
    image = addTScans(image)
                                    
    return image




In [ ]:
def get_explanatories(i, row):
    
    date = row['index']
    date = dt.strptime(date, '%d/%m/%Y')
    start = dt.strftime(date, '%Y-%m-%d')
    end = dt.strftime(date + timedelta(days=1), '%Y-%m-%d')
    
    point = ee.Geometry.Point(row.lon, row.lat)
    
    image = s1_collection.create(
        region=point.buffer(2000), 
        start_date=start,
        end_date=end, 
        add_ND_ratio=False,
        speckle_filter='QUEGAN',
        radiometric_correction='TERRAIN',
        slope_correction_dict={'model': 'surface', 'dem': 'USGS/SRTMGL1_003', 'buffer': 50}, #'CGIAR/SRTM90_V4'
        db=True
    ).first()

    # if there is no image, skip
    if not image.getInfo():
        return

    # get metadata
    props = image.getInfo()['properties']
    row['scene_id'] = props['system:index']
    row['ee_time'] = image.date().getInfo()
    row['ee_geom'] = image.geometry().getInfo()
    orbit = props['orbitProperties_pass']
    row['orbit_direction'] = 1 if orbit == 'ASCENDING' else 2

    row['acq_date'] = row['scene_id'][17:25]

    image = addAux(image)
    #sample_feature = ee.Feature(point).set('date', image.date())
    #image = addGPM(sample_feature).select('precipitation', 'prec_3', 'prec_7', 'prec_30')

    # get actual data
    image_red = image.reduceRegion(
        reducer=ee.Reducer.toList(),\
        geometry=point.buffer(150),\
        maxPixels=1e13,\
        scale=100
    );


    data_dict = image_red.getInfo()
    
    bandlist = [
     'VV', 'VH', 'VVVH_ratio', 'angle', 'LIA', #'layover', 'shadow', 'no_data_mask', 
     #'precipitation', 'prec_3', 'prec_7', 'prec_30', 
     #'gldas_mean', 'gldas_stddev', 'sm_gldas', 'sm_3', 'sm_7', 'sm_30',
     'elevation', 'aspect', 'slope', 
     'landcover',
     'kVV_mean', 'kVV_stdDev', 'kVV_p5', 'kVV_p95', 'kVH_mean', 'kVH_stdDev', 'kVH_p5', 'kVH_p95', 
     'VV_mean', 'VV_stdDev', 'VV_p5', 'VV_p95', 'VH_mean', 'VH_stdDev', 'VH_p5', 'VH_p95'
    ]

    for band in bandlist:
        if band == 'landcover':
            counts = np.bincount(data_dict[band])
            row[band] = np.argmax(counts)
        else:
            row[band] = np.mean(data_dict[band])

    d = {}
    d[i] = row
    df = pd.DataFrame.from_dict(d, orient='index')
    df.to_pickle(f'/home/vollrath/Indonesia_sm/samples_all/{i}.gpm.pickle')
    

In [5]:
from multiprocessing import Process
j = 29730

with tqdm(initial=j, total=len(gdf), file=sys.stdout) as pbar:
    for i, row in gdf.iterrows():
        
        if i >= j:
            #get_explanatories(i, row)
            #break
        

            ## https://medium.com/@chaoren/how-to-timeout-in-python-726002bf2291
            ec = None
            while ec is None != 0:
                p1 = Process(target=get_explanatories, args=(i, row, ), name='Process')
                p1.start()
                p1.join(timeout=180)
                p1.terminate()
                ec = p1.exitcode
            #
            pbar.set_description('processed: %d' % (i))
            pbar.update(1)
            

processed: 30315: 100%|██████████| 30316/30316 [22:49<00:00,  2.34s/it]  


In [11]:
for i, file in enumerate(Path('/home/vollrath/Indonesia_sm/samples_all/').glob('*.gpm.pickle')):
#    print(file)
    if i == 0:
        df = pd.read_pickle(file)
    else:
        df = pd.concat([df, pd.read_pickle(file)])

In [14]:
df.columns

Index(['station', 'index', 'location', 'province', 'lon', 'lat', 'GWL_max',
       'GWL_min', 'GWL_rata', 'SM_max', 'SM_min', 'SM_rata', 'Total',
       'geometry', 'scene_id', 'ee_time', 'ee_geom', 'orbit_direction',
       'acq_date', 'VV', 'VH', 'VVVH_ratio', 'angle', 'LIA', 'elevation',
       'aspect', 'slope', 'landcover', 'kVV_mean', 'kVV_stdDev', 'kVV_p5',
       'kVV_p95', 'kVH_mean', 'kVH_stdDev', 'kVH_p5', 'kVH_p95', 'VV_mean',
       'VV_stdDev', 'VV_p5', 'VV_p95', 'VH_mean', 'VH_stdDev', 'VH_p5',
       'VH_p95'],
      dtype='object')

In [15]:
df_no_nans = df.dropna()
print(len(df))
print(len(df_no_nans))
gdf = gpd.GeoDataFrame(df_no_nans, geometry=gpd.points_from_xy(df_no_nans.lon, df_no_nans.lat))
gdf.to_file('/home/vollrath/Indonesia_sm/samples_all/combined_s1_extract.gpkg', driver='GPKG')

3197
2928


In [5]:
c = gpd.read_file('/home/vollrath/Indonesia_sm/samples_all/combined_s1_extract.gpkg')
len(c)

2928